# Предобработка данных

### Загрузка 

In [1]:
from langchain_community.document_loaders import PyPDFLoader

loader_1 = PyPDFLoader(
    "../data/standard_data/sql_primer.pdf",
)

loader_2 = PyPDFLoader(
    "../data/standard_data/Tutorial_PostgreSQL.pdf",
)

### Соединение данных

In [2]:
docs = loader_1.load()
docs += loader_2.load()

print(len(docs)) # разбиене на веторы требует больших мощностей, в связи с чем обез данных
docs = docs[:10]

### Обработка данных от html и прочих символов

Обработка не доделана с ней надо подробне разбраться, смотреть как работает.
* у нас есть код, смотрите чтоб его не снесли

In [47]:
import spacy
from bs4 import BeautifulSoup

nlp = spacy.load("ru_core_news_sm")  # Замените на нужную модель

# Функция для очистки текста
def clean_text(doc):
    # Удаление HTML-тегов с помощью BeautifulSoup
    soup = BeautifulSoup(doc.page_content, "html.parser")
    clean_text = soup.get_text()

    # Обработка текста с помощью spaCy
    doc_spacy = nlp(clean_text)

    # Удаление лишних символов и создание очищенного текста
    cleaned_tokens = [token.text for token in doc_spacy if not token.is_punct and not token.is_space]
    cleaned_text = " ".join(cleaned_tokens)

    return cleaned_text

# Применение функции очистки к документам
cleaned_content = []
for document in docs:
    cleaned_content.append(clean_text(document))



### Перевод в формат для векторизации

Перевод в другой формат для вектаризации.
Вообще скорее всего переводт было не обязательно, достаточно найт нормальный метод из предложенных разделения на чанки text_splitter  
https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/

тут возникает проблема
1) если не переводить в формат Document то есть проблемы с чанками которую можно решить поменяв разделитель, читайте инф по ссылке
2) есили всёже переводить что я делаю, появляюся лишние символы, множество точек, из - за чего чанки становятся бессмысленными просмотрите
    * cleaned_content - плохо но прдобработаны
    * documents - получившийся из них с точками /н и подобной ересью 

нужно разобраться, как разбить на чанки не потеряв обработку данных на чистоту

In [24]:
from langchain_core.documents import Document

documents = []

# Проходим по каждой строке в cleaned_content
i = 0
for content in cleaned_content:
    # Создаем новый документ для каждой строки
    i += 1
    documents.append(Document(metadata={'source': 'source_info', 'page': i}, page_content=content))


### Разбиение на чанки

нужно поэксперементировать с данными и просмотреть чанки, на данный момент они несут практически 0 смысл


In [45]:
from langchain.text_splitter import CharacterTextSplitter
from langchain_core.documents import Document


text_splitter = CharacterTextSplitter(
    chunk_size=250,  # Максимальный размер части
    chunk_overlap=0  # Перекрытие между частями
)

# Разделяем содержимое каждого документа
split_documents = []
for doc in documents:
    split_parts = text_splitter.split_text(doc.page_content)
    for part in split_parts:
        split_documents.append(Document(metadata=doc.metadata, page_content=part))


### Векторизация

требует больших мощностей, нужно запускать с мощьного пк, посмотреть возможность работы на видюхе, поспрашивайте гпт

In [53]:
# создаём путь до места хранения
from pathlib import Path

# Получаем текущий рабочий каталог
current_dir = Path.cwd()

# Создаем путь к папке processed_data, которая находится на уровне выше текущего каталога
processed_data_path = current_dir.parent / 'data' / 'processed_data' / 'union.parquet'


In [54]:
from langchain_community.vectorstores import SKLearnVectorStore
from langchain_ollama import OllamaEmbeddings

# Create embeddings for documents and store them in a vector store
vectorstore = SKLearnVectorStore.from_documents(
    documents=split_documents[:5],
    embedding = OllamaEmbeddings(model="llama3.1"),
    persist_path= processed_data_path,  # for save
    serializer="parquet" # for save
)
retriever = vectorstore.as_retriever(k=4)

### Сохранение

In [55]:
# save
vectorstore.persist()